## ASSIGNMENT-2

### Name: Harsh Kashyap
Roll No. : 101917088
Batch: CSE4

In [ ]:
import cv2


faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

face_id = input('enter user id ')
print('\n[INFO] Initialising train data collection. Look at the camera for 120 seconds...')


count=0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    #drawing face detection annotations
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5,     
    )
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        count+=1

        #save sample images for training data
        cv2.imwrite("dataset/User."+str(face_id)+'.'+str(count)+".jpg", gray[y:y+h, x:x+w])
        cv2.imshow('Image', img)
        
    # # cv2.imshow("Webcam Input",img)
    # cv2.imshow("Face detection", cv2.flip(img, 1))

    k= cv2.waitKey(100) & 0xff
    if k==27:
        break
    elif count>=120:
        break
print('\n[INFO] Dataset created!!')


print('\n[INFO] Initialising test data collection. Look at the camera for 30 seconds...')


count=0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    #drawing face detection annotations
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5,     
    )
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        count+=1

        #save sample images for training data
        cv2.imwrite("test_data/User."+str(face_id)+'.'+str(count)+".jpg", gray[y:y+h, x:x+w])
        cv2.imshow('Image', img)
        
    # cv2.imshow("Webcam Input",img)
    cv2.imshow("Face detection", cv2.flip(img, 1))

    k= cv2.waitKey(100) & 0xff
    if k==27:
        break
    elif count>=30:
        break
print('\n[INFO] Dataset created!!')



cap.release()
cv2.destroyAllWindows()
# print('\n[INFO] Training data... Please Wait..')
# exec(open("face_recog_trainer.py").read())

Name: Harsh Kashyap
Roll No. : 101917088
Batch: CSE4

In [2]:
import cv2
import numpy as np
import os
from sklearn.metrics import confusion_matrix,accuracy_score
import pandas as pd


recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "Cascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX
l=[]
l1=[1]*30
def draw_border(img, point1, point2, point3, point4, line_length):

    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    x4, y4 = point4    

    cv2.circle(img, (x1, y1), 3, (255, 0, 255), -1)    #-- top_left
    cv2.circle(img, (x2, y2), 3, (255, 0, 255), -1)    #-- bottom-left
    cv2.circle(img, (x3, y3), 3, (255, 0, 255), -1)    #-- top-right
    cv2.circle(img, (x4, y4), 3, (255, 0, 255), -1)    #-- bottom-right

    cv2.line(img, (x1, y1), (x1 , y1 + line_length), (0, 255, 0), 2)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length , y1), (0, 255, 0), 2)

    cv2.line(img, (x2, y2), (x2 , y2 - line_length), (0, 255, 0), 2)  #-- bottom-left
    cv2.line(img, (x2, y2), (x2 + line_length , y2), (0, 255, 0), 2)

    cv2.line(img, (x3, y3), (x3 - line_length, y3), (0, 255, 0), 2)  #-- top-right
    cv2.line(img, (x3, y3), (x3, y3 + line_length), (0, 255, 0), 2)

    cv2.line(img, (x4, y4), (x4 , y4 - line_length), (0, 255, 0), 2)  #-- bottom-right
    cv2.line(img, (x4, y4), (x4 - line_length , y4), (0, 255, 0), 2)

    return img
#id counter 
id=0
line_length = 30
#for saving captured images
img_counter=0
names = ['None', 'ABHINAV']
# print('\n[INFO] Starting Camera.. Please Wait..')
# cam = cv2.VideoCapture(0)
print('\n[TIP] For capturing an image.. Press Space Key.')
count=0
abhinav_count = 0
unknown_count=0
while count<30:
    img = cv2.imread("test_data/User.001" + '.' + str(count+1)+".jpg")
    img = cv2.flip(img,1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5)

    for(x,y,w,h) in faces:
        img = draw_border(img, (x,y), (x,y+h), (x+w,y), (x+w,y+h), line_length)
        # cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        # If confidence is less them 100 ==> "0" : perfect match 
        if (confidence < 50):
            abhinav_count+=1
            l.append(1)
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            # id = "unknown"
            unknown_count+=1
            l.append(0)
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y+10), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
        cv2.putText(
                    img, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   ) 
        
    
    cv2.imwrite("predicted_data/test_image."+str(count)+".jpg", img)
    count+=1
    # cv2.imshow('camera',img) 
    # k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    # if k == 27:
    #     break
    # elif k%256 == 32:
    #     # SPACE pressed
    #     img_name = "captured_img_{}.png".format(img_counter)
    #     cv2.imwrite("captured_images/"+img_name, img)
    #     print("{} saved!".format(img_name))
    #     img_counter+=1



# Do a bit of cleanup
print("\n[INFO] Exiting Program and cleanup stuff")
# cam.release()
cv2.destroyAllWindows()
m=len(l)
for i in range(m,30):
    l.append(1)
print(anubhav_count)
print(unknown_count)
print(l1)
print(l)
print(confusion_matrix(y_true=l1,y_pred=l))
print(accuracy_score(y_true=l1,y_pred=l))


[TIP] For capturing an image.. Press Space Key.

[INFO] Exiting Program and cleanup stuff
0
5
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[ 0  0]
 [ 5 25]]
0.8333333333333334


Name: Harsh Kashyap
Roll No. : 101917088
Batch: CSE4

THANK YOU